### Install Dependencies

### Google Colab Setup

In [ ]:
# from google.colab import drive
# #drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# !git clone https://github.com/optas/changeit3d
# %cd changeit3d
# !git submodule add https://github.com/ThibaultGROUEIX/ChamferDistancePytorch changeit3d/losses/ChamferDistancePytorch
# !pip install -e .
# !pip install Ninja
# !pip install wandb
# import sys
# sys.path.append('/content/drive/Shareddrives/CS766/changeit3d')

Cloning into 'changeit3d'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 184 (delta 12), reused 184 (delta 12), pack-reused 0
Receiving objects: 100% (184/184), 4.28 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/changeit3d
Cloning into '/content/changeit3d/changeit3d/losses/ChamferDistancePytorch'...
remote: Enumerating objects: 265, done.
remote: Total 265 (delta 0), reused 0 (delta 0), pack-reused 265
Receiving objects: 100% (265/265), 46.75 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (163/163), done.
Obtaining file:///content/changeit3d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
import torch
import numpy as np
import os.path as osp
from functools import partial

from changeit3d.in_out.changeit3d_net import prepare_input_data
from changeit3d.in_out.language_contrastive_dataset import LanguageContrastiveDataset
from changeit3d.in_out.pointcloud import pc_loader_from_npz, uniform_subsample
from changeit3d.in_out.basics import pickle_data
from changeit3d.in_out.basics import create_logger
from changeit3d.in_out.arguments import parse_evaluate_changeit3d_arguments

from changeit3d.utils.basics import parallel_apply
from changeit3d.models.model_descriptions import load_pretrained_changeit3d_net
from changeit3d.models.model_descriptions import load_pretrained_pc_ae

from changeit3d.evaluation.auxiliary import pc_ae_transform_point_clouds, sgf_transform_point_clouds
from changeit3d.external_tools.sgf.loader import initialize_and_load_sgf
from changeit3d.utils.visualization import visualize_point_clouds_3d_v2, plot_3d_point_cloud
from changeit3d.in_out.arguments import parse_evaluate_changeit3d_arguments
from changeit3d.in_out.changeit3d_net import prepare_input_data

import pandas as pd
from torch.utils.data import Dataset



%load_ext autoreload
%autoreload 2

: 

### Specify Data Paths to Utilize: This is specified using the excel `shapetalk_preprocessed_public_version_0.csv`

In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/CS766/changeit3d')

In [ ]:
##
## Manually specify the underlying files using our pretrained nets or yours
## COMPLETE - UPDATED WITH MY FILE PATHS
##

#shape_generator_type = "sgf"
shape_generator_type = "pcae"
#shape_generator_type = "resnet101"

top_data_dir = '../drive/Shareddrives/CS766/shapetalk_dataset'
top_train_dir = '../drive/Shareddrives/CS766/pretrained_models/data'
shape_talk_file = f'{top_data_dir}/shapetalk/language/shapetalk_preprocessed_public_version_0.csv'
vocab_file = f'{top_data_dir}/shapetalk/language/vocabulary.pkl'
top_pc_dir = f'{top_data_dir}/shapetalk/point_clouds/scaled_to_align_rendering'
pretrained_oracle_listener = f'{top_train_dir}/pretrained/listeners/oracle_listener/all_shapetalk_classes/rs_2023/listener_dgcnn_based/ablation1/best_model.pkl'
pretrained_shape_classifier =  f'{top_train_dir}/pretrained/pc_classifiers/rs_2022/all_shapetalk_classes/best_model.pkl'
shape_part_classifiers_top_dir = f'{top_train_dir}/pretrained/part_predictors/shapenet_core_based'
latent_codes_file = f'{top_train_dir}/pretrained/shape_latents/{shape_generator_type}_latent_codes.pkl'



### Loading in Pretrained Shapetalk PC-AE.
if shape_generator_type == "pcae":
    pretrained_shape_generator = f'{top_train_dir}/pretrained/pc_autoencoders/pointnet/rs_2022/points_4096/all_classes/scaled_to_align_rendering/08-07-2022-22-23-42/best_model.pt'
    ## selected PC-AE ablation:
    selected_ablation = 'decoupling_mag_direction/idpen_0.05_sc_True/' # decoupled and with self-contrast=True

### SGF-AE based ShapeTalk dataset pre-trained.
if shape_generator_type == "sgf":
    selected_ablation = 'decoupling_mag_direction/idpen_0.05_sc_True/'
    sub_sample_dataset = '100' # just evaluate over 100 randomly selected test shapes

# Pre-trained ChangeIT3D Arguments
pretrained_changeit3d = f'{top_train_dir}/pretrained/changers/{shape_generator_type}_based/all_shapetalk_classes/{selected_ablation}/best_model.pt'

notebook_arguments = []

notebook_arguments.extend(['-shape_talk_file', shape_talk_file])
notebook_arguments.extend(['-latent_codes_file', latent_codes_file])
notebook_arguments.extend(['-vocab_file', vocab_file])
notebook_arguments.extend(['-pretrained_changeit3d', pretrained_changeit3d])
notebook_arguments.extend(['-top_pc_dir', top_pc_dir])
notebook_arguments.extend(['--shape_generator_type', shape_generator_type])
notebook_arguments.extend(['--pretrained_oracle_listener', pretrained_oracle_listener])
notebook_arguments.extend(['--pretrained_shape_classifier', pretrained_shape_classifier])
notebook_arguments.extend(['--shape_part_classifiers_top_dir', shape_part_classifiers_top_dir])

if 'pretrained_shape_generator' in  locals():
    notebook_arguments.extend(['--pretrained_shape_generator', pretrained_shape_generator])

if 'sub_sample_dataset' in  locals():
    notebook_arguments.extend(['--sub_sample_dataset', sub_sample_dataset])


# Arguments being parsed here. #
args = parse_evaluate_changeit3d_arguments(notebook_arguments)
logger = create_logger(args.log_dir)



Input arguments:


{'batch_size': 1024,
 'clean_train_val_data': False,
 'compute_fpd': True,
 'evaluate_retrieval_version': False,
 'experiment_tag': None,
 'gpu_id': 0,
 'latent_codes_file': '../drive/Shareddrives/CS766/pretrained_models/data/pretrained/shape_latents/pcae_latent_codes.pkl',
 'log_dir': './logs',
 'n_sample_points': 2048,
 'num_workers': 10,
 'pretrained_changeit3d': '../drive/Shareddrives/CS766/pretrained_models/data/pretrained/changers/pcae_based/all_shapetalk_classes/decoupling_mag_direction/idpen_0.05_sc_True//best_model.pt',
 'pretrained_oracle_listener': '../drive/Shareddrives/CS766/pretrained_models/data/pretrained/listeners/oracle_listener/all_shapetalk_classes/rs_2023/listener_dgcnn_based/ablation1/best_model.pkl',
 'pretrained_shape_classifier': '../drive/Shareddrives/CS766/pretrained_models/data/pretrained/pc_classifiers/rs_2022/all_shapetalk_classes/best_model.pkl',
 'pretrained_shape_generator': '../drive/Shareddrives/CS766/pretrained_models/data/pretra

### Dataloader for Custom Shapes

*   Pass in same arguments as the normal `LanguageContrastiveDataset`



In [ ]:
# Use this for custom shapes
class ShapeTalkCustomShapeDataset(Dataset):
  def __init__(self, data_frame, to_stimulus_func=None, shuffle_items=False, n_distractors=2,
                 shape_to_latent_code=None, encoder=None, base_data_dir = None, target_base_dir = None):
        """
        Args:
            data_frame:
            to_stimulus_func:
            shuffle_items:
            n_distractors:
            shape_to_latent_code:
        """
        super(ShapeTalkCustomShapeDataset, self).__init__()
        self.df = data_frame
        self.shuffle_items = shuffle_items
        self.n_distractors = n_distractors
        self.to_stimulus_func = to_stimulus_func
        self.shape_to_latent_code = shape_to_latent_code
        self.encoder = encoder
        self.base_data_dir = base_data_dir
        self.target_base_dir = target_base_dir

        # self.remove_missing_data()

  # Remove Missing Data from the point clouds #
  def remove_missing_data(self):
      if self.base_data_dir is None or self.to_stimulus_func is not None:
          return
      bad_data_idxs = []

      # Working with a data frame #
      for idx in range(len(self.df)):
          item_ids = []
          row = self.df.iloc[idx]
          for i in range(1, self.n_distractors + 1):
              item_ids.append(row[f'distractor_{i}'])
          item_ids.append(row['target'])  #

          # Item Ids #
          for x in item_ids:
            item_class, item_dataset, noise_value, item_code = x.split("/")
            test_pc_path = self.base_data_dir + "/" + item_class + "/" + item_dataset + "/" + noise_value + "/" + item_code
            if not os.path.exists(test_pc_path):
                bad_data_idxs.append(idx)
                break

      print("Number of Bad Indexes", len(bad_data_idxs))
      self.df.drop(bad_data_idxs)
      print("Removed Missing Data, Dataset has", len(self.df), "elements")


  def __getitem__(self, index):
      row = self.df.iloc[index]
      tokens = row['tokens_encoded']
      tokens = np.array(tokens).T  # todo do via collate.

      item_ids = []
      for i in range(1, self.n_distractors + 1):
          item_ids.append(row[f'distractor_{i}'])

      item_ids.append(row['target'])  # now target is last.
      item_ids = np.array(item_ids)
      n_items = len(item_ids)
      label = n_items - 1


      if self.shuffle_items:
          idx = np.arange(n_items)
          np.random.shuffle(idx)
          item_ids = item_ids[idx]
          label = np.where(idx == label)[0][0]

      res = dict()
      res['tokens'] = tokens
      res['label'] = label
      res['index'] = index
      res['pc_stimulus'] = index
      # Loading all point clouds #
      if self.to_stimulus_func is None:
          res['pc_stimulus'] = []
          for i, x in enumerate(item_ids):
              if i == len(item_ids) - 1:
                 item_class, item_dataset, item_code = x.split("/")
                 test_pc_path = self.target_base_dir + "/" + item_class + "/" + item_dataset + "/" + item_code + ".npz"
              else:
                item_class, item_dataset, noise_value, item_code = x.split("/")
                test_pc_path = self.base_data_dir + "/" + item_class + "/" + item_dataset + "/" + noise_value + "/" + item_code

              res['pc_stimulus'].append(np.load(test_pc_path)['pointcloud'])

          res['pc_stimulus'] = np.stack(res['pc_stimulus'])
          # print(res['pc_stimulus'].shape)
          res['stimulus'] = index
      else:
          # print("Using Stimulus", self.to_stimulus_func)
          res['stimulus'] = []
          for x in item_ids:
              res['stimulus'].append(self.to_stimulus_func(x))
          res['stimulus'] = np.stack(res['stimulus'])
      return res

  def __len__(self):
      return len(self.df)



### Loading ChangeIt3D Net: ##



In [ ]:
import os
print(args)


Namespace(shape_talk_file='../drive/Shareddrives/CS766/shapetalk_dataset/shapetalk/language/shapetalk_preprocessed_public_version_0.csv', vocab_file='../drive/Shareddrives/CS766/shapetalk_dataset/shapetalk/language/vocabulary.pkl', latent_codes_file='../drive/Shareddrives/CS766/pretrained_models/data/pretrained/shape_latents/pcae_latent_codes.pkl', pretrained_changeit3d='../drive/Shareddrives/CS766/pretrained_models/data/pretrained/changers/pcae_based/all_shapetalk_classes/decoupling_mag_direction/idpen_0.05_sc_True//best_model.pt', top_pc_dir='../drive/Shareddrives/CS766/shapetalk_dataset/shapetalk/point_clouds/scaled_to_align_rendering', restrict_shape_class=['chair', 'table', 'lamp'], pretrained_shape_classifier='../drive/Shareddrives/CS766/pretrained_models/data/pretrained/pc_classifiers/rs_2022/all_shapetalk_classes/best_model.pkl', compute_fpd=True, shape_part_classifiers_top_dir='../drive/Shareddrives/CS766/pretrained_models/data/pretrained/part_predictors/shapenet_core_based', 

In [ ]:
import os

# Get the current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")
new_dir = f'..{current_dir}'
print(new_dir)

# List all files and directories in the current directory
print("Directory contents:")
for item in os.listdir(current_dir):
    print(item)

# Check if the file exists
file_path = '../drive/Shareddrives/CS766/pretrained_models/data/pretrained/shape_latents/pcae_latent_codes.pkl'
absolute_file_path = os.path.abspath(file_path)
file_exists = os.path.isfile(absolute_file_path)
print(f"Does the file '{absolute_file_path}' exist? {file_exists}")

Current directory: /content/changeit3d
../content/changeit3d
Directory contents:
environment.yml
assets
.git
setup.py
.gitmodules
changeit3d.egg-info
README.md
changeit3d
logs
.gitignore
Does the file '/content/drive/Shareddrives/CS766/pretrained_models/data/pretrained/shape_latents/pcae_latent_codes.pkl' exist? True


In [ ]:
logger.info('Loading pretrained ChangetIt3DNet (C3DNet)')
df, shape_to_latent_code, shape_latent_dim, vocab = prepare_input_data(args, logger)

c3d_net, best_epoch, c3d_args = load_pretrained_changeit3d_net(args.pretrained_changeit3d, shape_latent_dim, vocab)
if torch.cuda.is_available():
  device = torch.device("cuda:" + str(args.gpu_id))
else:
  device = torch.device('cpu')

c3d_net = c3d_net.to(device)
logger.info(f'The model is variant `{c3d_args.shape_editor_variant}` trained with {c3d_args.identity_penalty} identity penalty and Self-Contrast={c3d_args.self_contrast}.')
logger.info(f'Loaded at epoch {best_epoch}.')

INFO:root:Loading pretrained ChangetIt3DNet (C3DNet)


Loading pretrained ChangetIt3DNet (C3DNet)


INFO:root:Latent codes with dimension 256 are loaded.


Latent codes with dimension 256 are loaded.


INFO:root:Restricting to class(es) ['chair', 'table', 'lamp']. Total utterances: 217747


Restricting to class(es) ['chair', 'table', 'lamp']. Total utterances: 217747
Doing ST ablation decoupling_mag_direction with self contrast True


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
INFO:root:The model is variant `decoupling_mag_direction` trained with 0.05 identity penalty and Self-Contrast=True.


The model is variant `decoupling_mag_direction` trained with 0.05 identity penalty and Self-Contrast=True.


INFO:root:Loaded at epoch 77.


Loaded at epoch 77.


###  Loading the Input Data and PC-AE

In [ ]:
from argparse import ArgumentParser
import json
from changeit3d.models.point_net import PointNet
from changeit3d.models.mlp import MLP
from changeit3d.models.pointcloud_autoencoder import PointcloudAutoencoder
from changeit3d.in_out.basics import load_state_dicts

def read_saved_args(config_file, override_or_add_args=None, verbose=False):
    """
    :param config_file: json file containing arguments
    :param override_args: dict e.g., {'gpu': '0'} will set the resulting arg.gpu to be 0
    :param verbose:
    :return:
    """
    parser = ArgumentParser()
    args = parser.parse_args([])
    with open(config_file, 'r') as f_in:
        args.__dict__ = json.load(f_in)

    if override_or_add_args is not None:
        for key, val in override_or_add_args.items():
            args.__setattr__(key, val)



    return args

def describe_pc_ae(args):
    # Make an AE.
    if args.encoder_net == 'pointnet':
        ae_encoder = PointNet(init_feat_dim=3, conv_dims=args.encoder_conv_layers)
        encoder_latent_dim = args.encoder_conv_layers[-1]
    else:
        raise NotImplementedError()

    if args.decoder_net == 'mlp':
        ae_decoder = MLP(in_feat_dims=encoder_latent_dim,
                         out_channels=args.decoder_fc_neurons + [args.n_pc_points * 3],
                         b_norm=False)

    model = PointcloudAutoencoder(ae_encoder, ae_decoder)
    return model



def load_pretrained_pc_ae(model_file):
    config_file = osp.join(osp.dirname(model_file), 'config.json.txt')
    pc_ae_args = read_saved_args(config_file)
    pc_ae = describe_pc_ae(pc_ae_args)

    # if osp.join(pc_ae_args.log_dir, 'best_model.pt') != osp.abspath(model_file):
        # warnings.warn("The saved best_model.pt in the corresponding log_dir is not equal to the one requested.")

    if torch.cuda.is_available():
      best_epoch = load_state_dicts(model_file, model=pc_ae)
    else:
      best_epoch = load_state_dicts(model_file, model=pc_ae, map_location='cpu')
    print(f'Pretrained PC-AE is loaded at epoch {best_epoch}.')
    return pc_ae, pc_ae_args

In [ ]:

if args.shape_generator_type == 'pcae':
    pc_ae, pc_ae_args = load_pretrained_pc_ae(args.pretrained_shape_generator)
    pc_ae = pc_ae.to(device)
    pc_ae = pc_ae.eval()
elif args.shape_generator_type == 'sgf':
    sgf_ae = initialize_and_load_sgf()
    sgf_trainer = sgf_ae.trainer
    sgf_trainer.cfg.inference.num_points = args.n_sample_points
else:
    assert False


def to_stimulus_func(x):
    return shape_to_latent_code[x]

split = 'test'
ndf = df[df.changeit_split == split].copy()
ndf.reset_index(inplace=True, drop=True)

if args.sub_sample_dataset is not None:
    np.random.seed(args.random_seed)
    ndf = ndf.sample(args.sub_sample_dataset)
    ndf.reset_index(inplace=True, drop=True)

dataset = ShapeTalkCustomShapeDataset(ndf,
                                     to_stimulus_func,
                                     n_distractors=1,
                                     shuffle_items=False)  # important, source (distractor) now is first

dataloader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=args.batch_size,
                                         shuffle=False,
                                         num_workers=args.num_workers,
                                         worker_init_fn=lambda _ : np.random.seed(args.random_seed))

Pretrained PC-AE is loaded at epoch 186.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## a)  Language Based Evaluation

- Run the experiment on varying formats of input data and compare evaluation performance.



In [ ]:
from changeit3d.language.vocabulary import Vocabulary
from ast import literal_eval
import os

language_file_dict = {
    'public_version': f'{top_data_dir}/shapetalk/language/shapetalk_preprocessed_public_version_0.csv',
    'modified': f'../drive/Shareddrives/CS766/no_shade_utterances_modified.csv'
    #'modified': f'{top_data_dir}/shapetalk/language/shapetalk_preprocessed_filtered.csv'
    # folder to remove parts
    #'modified': f'../../drive/Shareddrives/CS766/shapetalk_preprocessed_filtered_for_shape_removal.csv'
    # folder of noisey files
    #'modified': f'../../drive/Shareddrives/CS766/custom_noise_dataset_v4.csv'
    # should I make a file where the cropped shapes are the source and the OG ones are the target with our own utterances? still need help with token_embedding

}

output_folder = f"{top_data_dir}/../generation_results/language/"

def preprocess_and_load_df(args, df_file_name):
  cur_df = pd.read_csv(df_file_name)

  cur_df.tokens_encoded = cur_df.tokens_encoded.apply(literal_eval)
  vocab = Vocabulary.load(args.vocab_file)

  if hasattr(args, "add_shape_glot") and args.add_shape_glot:
      raise NotImplementedError('Not in public version')
      cur_df = add_sg_to_snt(cur_df, vocab, args.split_file)

  cur_df = cur_df.assign(target=df.target_uid)
  cur_df = cur_df.assign(distractor_1=df.source_uid)

    # constrain training in language of particular classes
  # if len(args.restrict_shape_class) > 0:
  #     mask = cur_df.target_object_class.isin(set(args.restrict_shape_class))
  #     cur_df = df[mask].copy()
  #     cur_df.reset_index(inplace=True, drop=True)

  #     msg = 'Restricting to class(es) {}. Total utterances: {}'.format(args.restrict_shape_class, len(cur_df))
  #     print(msg)

  return cur_df

for lang_exp_type, exp_file_name in language_file_dict.items():

    cur_df = preprocess_and_load_df(args, exp_file_name)
    mask = cur_df.target_object_class.isin(set(args.restrict_shape_class))
    cur_df = cur_df[mask].copy()
    cur_df.reset_index(inplace=True, drop=True)

    msg = 'Restricting to class(es) {}. Total utterances: {}'.format(args.restrict_shape_class, len(cur_df))
    cur_dataset = ShapeTalkCustomShapeDataset(cur_df,
                                     to_stimulus_func,
                                     n_distractors=1,
                                     shuffle_items=False)  # important, source (distractor) now is first

    cur_dataloader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=args.batch_size,
                                         shuffle=False,
                                         num_workers=args.num_workers,
                                         worker_init_fn=lambda _ : np.random.seed(args.random_seed))


    if args.shape_generator_type == 'pcae':
      transformation_results = pc_ae_transform_point_clouds(pc_ae,
                                                            c3d_net,
                                                            cur_dataloader,
                                                            stimulus_index=0,
                                                            scales=[0, 1],  # use "0" to get also the simple reconstruction of the decoder (no edit)
                                                            device=device)

    elif args.shape_generator_type == 'sgf':
      transformation_results = sgf_transform_point_clouds(sgf_trainer,
                                                          c3d_net,
                                                          cur_dataloader,
                                                          stimulus_index=0,
                                                          scales=[1],
                                                          n_points_per_shape=args.n_sample_points,
                                                          batch_size=128,
                                                          max_recon_batches=None)


    np.save(os.path.join(output_folder, f"{args.shape_generator_type}_dataset_{lang_exp_type}.npy"), transformation_results)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## b) Noisy Point Cloud Evaluation
- Pass in the noisy point clouds along with the original language instructions
- Compare this with varying levels of noise.

In [2]:
from collections import defaultdict

@torch.no_grad()
def pc_ae_transform_point_clouds(
    pc_ae,
    direction_finder,
    data_loader,
    stimulus_index,
    scales=[1],
    device="cuda",
    encode_stimulus = False
  ):
    """

    Args:
        pc_ae:
        direction_finder:
        data_loader:
        stimulus_index: stimulus location in data_loader (e.g., is it the first or the second shape)
        scales: use them to multiply the edit_latent before you add it to the original latent, this way you can *manually* boost or attenuate the edit's effect
        device:

    Returns:
        Let's assume that:
            1) the input language/shape-dataset concerns the transformation for N shape-language pairs.
            2) the latent space is L-dimensional

        then,

         A dictionary carrying the following items:
            'z_codes' -> dict, carrying the updated *final* latent codes. The keys are the input magnitudes. Each value is
                an N x L numpy array.
            'recons' -> dict, the N decoded/reconstructed point-clouds. The keys are input magnitudes. Each value is
                an N x PC-points x 3 numpy array.
            'tokens' -> list of lists, the N sentences used to create the transformations
            'edit_latents' -> N x L numpy array, the latents corresponding to the edits (before adding them to each input)
            'magnitudes' -> N x 1 numpy array, carrying the magnitudes the editing network guessed for each input.
    """

    results = get_transformed_latent_code(pc_ae, direction_finder, data_loader, stimulus_index, scales=scales, device=device, encode_stimulus = encode_stimulus)

    pc_ae.eval()
    all_recons = defaultdict(list)

    for key, val in results['z_codes'].items():
        recons = pc_ae.decoder(torch.from_numpy(val).to(device))
        recons = recons.view([len(recons), -1, 3]).cpu()
        all_recons[key].append(recons)

    for key in all_recons:
        all_recons[key] = torch.cat(all_recons[key]).numpy()

    results['recons'] = all_recons
    return results

@torch.no_grad()
def get_transformed_latent_code(pc_ae, direction_finder, data_loader, stimulus_index, scales=[1], device="cuda", encode_stimulus = False):
    """  Extract transformation for a given latent code based on LatentDirectionFinder
    """
    direction_finder.eval()

    all_z_codes = defaultdict(list)
    all_tokens = []
    all_edit_latents = []
    all_magnitudes = []

    # Enumerating the batches of data loader #
    for batch in data_loader:
        t = batch['tokens'].to(device)

        # Should I encode the stimulus function? #
        if encode_stimulus:
          batch_size, n_distractors, num_points, d = batch['pc_stimulus'].shape
          ae_input = batch['pc_stimulus'].view(-1, num_points, d).permute(0, 2, 1).to(device)
          batch['stimulus'] = pc_ae.encoder(ae_input.float())
          batch['stimulus'] = batch['stimulus'].view(batch_size, n_distractors, -1)

        s = batch['stimulus'][:, stimulus_index].to(device)
        edit_latent, guessed_mag = direction_finder(t, s)
        # print(edit_latent, guessed_mag)

        for scale in scales:
            if scale == 0:  # no transformation, just return the input/starting latent code
                transformed = s
            else:
                transformed = s + scale * edit_latent

            all_z_codes[scale].append(transformed.cpu())
        all_edit_latents.append(edit_latent.cpu())
        all_magnitudes.append(guessed_mag.cpu())

        all_tokens.extend(t.tolist())


    all_edit_latents = torch.cat(all_edit_latents).numpy()
    all_magnitudes = torch.cat(all_magnitudes).numpy()

    for scale in scales:
        all_z_codes[scale] = torch.cat(all_z_codes[scale]).numpy()

    results = {'z_codes': all_z_codes,
               'tokens': all_tokens,
               'edit_latents': all_edit_latents,
               'magnitudes': all_magnitudes}

    return results

def load_custom_df(shape_talk_file, args):
    df = pd.read_csv(shape_talk_file)
    df.tokens_encoded = df.tokens_encoded.apply(literal_eval)
    vocab = Vocabulary.load(args.vocab_file)

    if hasattr(args, "add_shape_glot") and args.add_shape_glot:
        raise NotImplementedError('Not in public version')
        df = add_sg_to_snt(df, vocab, args.split_file)

    df = df.assign(target=df.target_uid)
    df = df.assign(distractor_1=df.source_uid)

    # if len(args.restrict_shape_class) > 0:
        # mask = df.target_object_class.isin(set(args.restrict_shape_class))
        # df = df[mask].copy()
        # df.reset_index(inplace=True, drop=True)

    return df

NameError: name 'torch' is not defined

### Obtain Transformed Points

In [1]:
output_folder = f"{top_data_dir}/../generation_results/noisy_point_clouds"
os.makedirs(output_folder, exist_ok = True)

sigma_list = ['0.01', '0.02', '0.03']
categories = ['mug', 'lamp', 'bottle']
pc_data_path = f"{top_data_dir}/../noise_added_to_point_clouds/"
target_base_dir = f"{top_data_dir}/shapetalk/point_clouds/scaled_to_align_rendering"
csv_out_path = f"{top_data_dir}/../noisy_data.csv"
exp_type = "noisy_pc"

for sigma in sigma_list:
  for category in categories:
    shape_talk_file = pc_data_path + "/"  + category + "_noise" + "Sigma" + sigma.split('.')[1] + "_exp1.csv"
    cur_df = load_custom_df(shape_talk_file, args)
    print(f"Running {sigma} on the Category: {category}")

    msg = 'Restricting to class(es) {}. Total utterances: {}'.format(args.restrict_shape_class, len(cur_df))
    cur_dataset = ShapeTalkCustomShapeDataset(
                                      cur_df,
                                      base_data_dir=pc_data_path,
                                      target_base_dir=target_base_dir,
                                      to_stimulus_func=None,
                                      n_distractors=1,
                                      shuffle_items=False)  # important, source (distractor) now is first

    cur_dataloader = torch.utils.data.DataLoader(dataset=cur_dataset,
                                            batch_size=8,
                                            shuffle=False,
                                            num_workers=1,
                                            worker_init_fn=lambda _ : np.random.seed(args.random_seed))


    cur_dataset.df.to_csv(csv_out_path)
    full_df = cur_dataset.df
    if args.shape_generator_type == 'pcae':
        transformation_results = pc_ae_transform_point_clouds(pc_ae,
                                                              c3d_net,
                                                              cur_dataloader,
                                                              stimulus_index=0,
                                                              scales=[0, 1],  # use "0" to get also the simple reconstruction of the decoder (no edit)
                                                              device=device,
                                                              encode_stimulus=True)

    elif args.shape_generator_type == 'sgf':
        transformation_results = sgf_transform_point_clouds(sgf_trainer,
                                                            c3d_net,
                                                            cur_dataloader,
                                                            stimulus_index=0,
                                                            scales=[1],
                                                            n_points_per_shape=args.n_sample_points,
                                                            batch_size=128,
                                                            max_recon_batches=None)

    np.save(os.path.join(output_folder, f"{args.shape_generator_type}_dataset_{exp_type}_{sigma}_{category}.npy"), transformation_results)


NameError: name 'top_data_dir' is not defined

### c) Segmented Point Clouds:
- Show that using segmentation of parts in the target along with an input which is used for the rest of the experiments
- Using the language instruction to segment the point clouds (Missing arms)/Legs

In [ ]:
output_folder = f"{top_data_dir}/../generation_results/noisy_point_clouds"
os.makedirs(output_folder, exist_ok = True)

sigma_list = ['0.01', '0.02', '0.03']
categories = ['vase', 'mug', 'lamp', 'bottle']
pc_data_path = f"{top_data_dir}/../noise_added_to_point_clouds/"
target_base_dir = f"{top_data_dir}/shapetalk/point_clouds/scaled_to_align_rendering"
csv_out_path = f"{top_data_dir}/../noisy_data.csv"
exp_type = "noisy_pc"

shape_talk_file = pc_data_path + "/"  + category + "_noise" + "Sigma" + sigma.split('.')[1] + "_exp1.csv"
cur_df = load_custom_df(shape_talk_file, args)
print(f"Running {sigma} on the Category: {category}")

cur_dataset = ShapeTalkCustomShapeDataset(
                                  cur_df,
                                  base_data_dir=pc_data_path,
                                  target_base_dir=target_base_dir,
                                  to_stimulus_func=None,
                                  n_distractors=1,
                                  shuffle_items=False)  # important, source (distractor) now is first

cur_dataloader = torch.utils.data.DataLoader(dataset=cur_dataset,
                                            batch_size=8,
                                            shuffle=False,
                                            num_workers=1,
                                            worker_init_fn=lambda _ : np.random.seed(args.random_seed))

    cur_dataset.df.to_csv(csv_out_path)
    full_df = cur_dataset.df
    # Point Cloud Auto-Encoder #
    if args.shape_generator_type == 'pcae':
        transformation_results = pc_ae_transform_point_clouds(pc_ae,
                                                              c3d_net,
                                                              cur_dataloader,
                                                              stimulus_index=0,
                                                              scales=[0, 1],  # use "0" to get also the simple reconstruction of the decoder (no edit)
                                                              device=device,
                                                              encode_stimulus=True)
    # If the shape generator is SGF #
    elif args.shape_generator_type == 'sgf':
        transformation_results = sgf_transform_point_clouds(sgf_trainer,
                                                            c3d_net,
                                                            cur_dataloader,
                                                            stimulus_index=0,
                                                            scales=[1],
                                                            n_points_per_shape=args.n_sample_points,
                                                            batch_size=128,
                                                            max_recon_batches=None)

    np.save(os.path.join(output_folder, f"{args.shape_generator_type}_dataset_{exp_type}_{sigma}_{category}.npy"), transformation_results)


Using Stimulus None


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-106-a54072ea9d87>", line 78, in __getitem__
    test_pc_path = self.base_data_dir + "/" + item_class + "/" + item_code + ".npz"
TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'


In [ ]:
data_test.keys()

dict_keys(['tokens', 'label', 'index', 'stimulus'])

In [ ]:
cur_dataset.shape_to_latent_code